In [1]:
import tweepy
from tweepy.auth import OAuthHandler
import dask.dataframe as dd
from dask.multiprocessing import get

consumer_key = 'TD1vDc5ESsUWICKCqEmRqwiXY'
consumer_secret = 'anldfA300NKL2ly2sO9EOBFI82h51Xw5dlpNy7JnEvh2WQHRml'
access_token= '1064407182505009153-4rgPNk0Psv2coTWbuH5ssdbyIWLYC3'
access_token_secret = 'gRyY5dZdYhd70jHtA6QsArK7otEZ34miTjo5BUv0hrbFT'
auth = tweepy.auth.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.auth.API(auth)

#for tweet in public_tweets:
#    print(tweet.text)

In [2]:
user = api.get_user(id = 8729092)

In [5]:
user.screen_name

'sunjournal'

In [6]:
user.screen_name

'sunjournal'

In [7]:
import numpy as np
import pandas as pd

In [8]:
rawdata_polluters = pd.read_csv('datasets/content_polluters.txt',sep='\t', usecols=[0])
rawdata_legimaters = pd.read_csv('datasets/legitimate_users.txt',sep='\t',  usecols=[0])

In [9]:
rawdata_polluters.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22223 entries, 0 to 22222
Data columns (total 1 columns):
UserID    22223 non-null int64
dtypes: int64(1)
memory usage: 173.7 KB


In [10]:
rawdata_legimaters.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19275 entries, 0 to 19274
Data columns (total 1 columns):
614    19275 non-null int64
dtypes: int64(1)
memory usage: 150.7 KB


In [11]:
rawdata_polluters.columns=['user_id']
rawdata_legimaters.columns=['user_id']

In [12]:
rawdata_polluters[:5]

,user_id
0,6301
1,10836
2,10997
3,633293
4,717883


In [13]:
rawdata_legimaters[:5]

,user_id
0,1038
1,1437
2,2615
3,3148
4,5593


In [14]:
user_list = pd.concat([rawdata_polluters,rawdata_legimaters],axis=0)

In [16]:
user_list.reset_index()

,index,user_id
0,0,6301
1,1,10836
2,2,10997
3,3,633293
4,4,717883
5,5,763068
6,6,783705
7,7,788352
8,8,810011
9,9,815299


In [17]:
def splitDataFrameIntoSmaller(df, chunkSize = 10000): 
    listOfDf = []
    numberChunks = len(df) // chunkSize + 1
    for i in range(numberChunks):
        listOfDf.append(df[i*chunkSize:(i+1)*chunkSize])
    return listOfDf

In [18]:
user_list_split = splitDataFrameIntoSmaller(user_list, chunkSize = 100)

In [19]:
result = []
for each in user_list_split:
    id_list = each['user_id'].to_list()
    retrieved_json = api.lookup_users(user_ids=id_list)
    for item in retrieved_json:
        result.append(item)

In [37]:
df = pd.DataFrame(columns=['user_id', 'screen_name'])

In [39]:
for index,each in enumerate(result):
    df.at[index,'user_id'] = each.id
    df.at[index,'screen_name']=each.screen_name

In [40]:
df

,user_id,screen_name
0,6301,msjacbowie
1,10836,rex_huang
2,10997,philcampbell
3,633293,iboughtamac
4,717883,djdick
5,763068,vuchuu
6,783705,davepeck
7,788352,miguelcarrasco
8,810011,ChicagoWebDev
9,815299,benphoster


In [41]:
export = df.to_csv(r'id_screenName_list.csv',index = None, header=True)

### Note: many suspended account have been removed by twitter, so we cant find their screen_name from user_id. The sample size becomes small in this way.